In [1]:
import os
import sys
import bs4 as bs
import pandas as pd
import glob
import re

### D'abord, on import les métadonnées pour les rendre exploitables par BS4

In [2]:
meta = pd.read_csv("metadata.tsv", "\t") # Importation des métadonnées précédemment collectées

/tmp/ipykernel_92393/829467055.py:1: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  meta = pd.read_csv("metadata.tsv", "\t") # Importation des métadonnées précédemment collectées


In [3]:
files = glob.glob("./xml/*.xml") # On liste les fichiers

In [4]:
list_ark = [] # On isoles les id ARK propres aux fichiers présents dans le dossier
for i in files:
    if re.search(r"(bpt\w*)", i):
        list_ark.append(re.search(r"(bpt\w*)", i).group(1))

In [5]:
len(list_ark)

In [6]:
for ark in list_ark:

    # Import des métadonnées pour les rendre exploitables
    
    df = meta.loc[meta['Ark'].str.contains(ark)]
    df = df.drop('Unnamed: 0', axis=1)
    df2 = df.loc[df['Ark'].str.contains(ark)]
    liste_elements = df2.iloc[0].values.flatten().tolist()
    
    # Création de la racine et des balises du teiHeader principales
    
    soup = bs.BeautifulSoup(features='xml')
    soup.append(soup.new_tag('TEI', xmlns="http://www.tei-c.org/ns/1.0"))
    
    soup.TEI.append(soup.new_tag('teiHeader'))

    soup.TEI.teiHeader.append(soup.new_tag('fileDesc'))
    soup.TEI.teiHeader.append(soup.new_tag('encodingDesc'))
    soup.TEI.teiHeader.append(soup.new_tag('profileDesc'))

    soup.TEI.teiHeader.fileDesc.append(soup.new_tag('titleStmt'))
    soup.TEI.teiHeader.fileDesc.append(soup.new_tag('publicationStmt'))
    soup.TEI.teiHeader.fileDesc.append(soup.new_tag('sourceDesc'))

    soup.TEI.teiHeader.encodingDesc.append(soup.new_tag('editorialDecl'))
    soup.TEI.teiHeader.encodingDesc.append(soup.new_tag('projectDesc'))

    soup.TEI.teiHeader.profileDesc.append(soup.new_tag('langUsage'))
    
    #On s'occupe du fileDesc

        # D'abord, le titleStmt

    new = soup.new_tag('title', type="main")
    new.string = "A Muvra"
    soup.TEI.teiHeader.fileDesc.titleStmt.append(new)

    new = soup.new_tag('title', type="sub")
    new.string = "Giurnale di e pieve di Corsica"
    soup.TEI.teiHeader.fileDesc.titleStmt.append(new)

    soup.TEI.teiHeader.fileDesc.titleStmt.append(soup.new_tag('author'))
    new = soup.new_tag('persName')
    new.string = "Petru Rocca"
    soup.TEI.teiHeader.fileDesc.titleStmt.author.append(new)

    soup.TEI.teiHeader.fileDesc.titleStmt.append(soup.new_tag('respStmt'))
    new = soup.new_tag('name')
    new.string = "Deborah Paci"
    soup.TEI.teiHeader.fileDesc.titleStmt.respStmt.append(new)
    new = soup.new_tag('orgName')
    new.string = "Università di Bologna | Università di Modena e Reggio Emilia"
    soup.TEI.teiHeader.fileDesc.titleStmt.respStmt.append(new)
    new = soup.new_tag('resp')
    new.string = "Supervision et inteprétation des résultats"
    soup.TEI.teiHeader.fileDesc.titleStmt.respStmt.append(new)

    soup.TEI.teiHeader.fileDesc.titleStmt.insert(3 ,soup.new_tag('respStmt'))
    new = soup.new_tag('name')
    new.string = "Vincent Sarbach-Pulicani"
    soup.TEI.teiHeader.fileDesc.titleStmt.respStmt.append(new)
    new = soup.new_tag('orgName')
    new.string = "École nationale des chartes - PSL | Università di Pisa"
    soup.TEI.teiHeader.fileDesc.titleStmt.respStmt.append(new)
    new = soup.new_tag('resp')
    new.string = "Encodage TEI, récolte et exploitation des données"
    soup.TEI.teiHeader.fileDesc.titleStmt.respStmt.append(new)

        # Ensuite, le publicationStmt

    new = soup.new_tag('distributor')
    new.string = "BnF (Bibliothèque nationale de France)"
    soup.TEI.teiHeader.fileDesc.publicationStmt.append(new)

    soup.TEI.teiHeader.fileDesc.publicationStmt.append(soup.new_tag("address"))
    list_str = ['Bibliothèque François Mitterand', 'Quai François Mauriac', '75706, Paris', 'France']
    for s in list_str:
        new = soup.new_tag('addrLine')
        new.string = s
        soup.TEI.teiHeader.fileDesc.publicationStmt.address.append(new)

    new = soup.new_tag('idno', type="ark")
    new.string = liste_elements[0]
    soup.TEI.teiHeader.fileDesc.publicationStmt.append(new)

    new = soup.new_tag('availability', status="free")
    soup.TEI.teiHeader.fileDesc.publicationStmt.append(new)

    new = soup.new_tag('p')
    new.string = liste_elements[4]
    soup.TEI.teiHeader.fileDesc.publicationStmt.availability.append(new)


        # Enfin, le sourceDesc

    soup.TEI.teiHeader.fileDesc.sourceDesc.append(soup.new_tag('bibl'))

    new = soup.new_tag('author')
    soup.TEI.teiHeader.fileDesc.sourceDesc.bibl.append(new)

    new = soup.new_tag('persName')
    new.string = "Petru Rocca"
    soup.TEI.teiHeader.fileDesc.sourceDesc.bibl.author.append(new)

    new = soup.new_tag('title', type="main")
    new.string = "A Muvra"
    soup.TEI.teiHeader.fileDesc.sourceDesc.bibl.append(new)

    new = soup.new_tag('title', type="sub")
    new.string = "Giurnale di e pieve di Corsica"
    soup.TEI.teiHeader.fileDesc.sourceDesc.bibl.append(new)

    new = soup.new_tag('distributor')
    new.string = "Stamparia di A Muvra"
    soup.TEI.teiHeader.fileDesc.sourceDesc.bibl.append(new)

    soup.TEI.teiHeader.fileDesc.sourceDesc.bibl.append(soup.new_tag("address"))
    list_str = ['38 Cours Grandval', 'Aiacciu, Corsica', 'France']
    for s in list_str:
        new = soup.new_tag('addrLine')
        new.string = s
        soup.TEI.teiHeader.fileDesc.sourceDesc.bibl.address.append(new)

    new = soup.new_tag('edition')
    new.string = "Numéro " + str(liste_elements[3]) + " | Année " + str(liste_elements[2])
    soup.TEI.teiHeader.fileDesc.sourceDesc.bibl.append(new)

    new = soup.new_tag('date')
    new.string = str(liste_elements[1])
    soup.TEI.teiHeader.fileDesc.sourceDesc.bibl.append(new)
    
    # On passe maintenant à l'encodingDesc

    soup.TEI.teiHeader.encodingDesc.editorialDecl.append(soup.new_tag('normalization'))

    new = soup.new_tag('p')
    new.string = "Le texte a été mis en minuscule. Normalisation des césures selon le contexte  et suppression des tirets en fin de ligne pour homogénéiser les mots. La ponctuation a également été supprimée pour diminuer les conséquences du bruit de l'OCR."
    soup.TEI.teiHeader.encodingDesc.editorialDecl.normalization.append(new)

    new = soup.new_tag('p')
    new.string = "Utilisation de Tesseract OCR (cos + fra + ita). Documents segmentés à partir de fichiers UZN obtenus grâce aux fichiers ALTO mis à disposition par Gallica. Il s'agit de la publication des transcriptions des numéros d'« A Muvra » disponibles sur la plateforme Gallica de la BnF. Ces documents servent d'abord au mémoire de recherche en « Humanités numériques » de Vincent Sarbach-Pulicani. Ils servent également au projet d'article pour la revue « Umanistica digitale », issu d'une collaboration entre Deborah Paci et Vincent Sarbach-Pulicani."
    soup.TEI.teiHeader.encodingDesc.projectDesc.append(new)
    
    # Enfin, on passe au profileDesc

    list_str = ["cos", "fra", "ita"]
    for i in list_str:
        new = soup.new_tag('language', ident=i)
        soup.TEI.teiHeader.profileDesc.langUsage.append(new)   

    # On crée d'abord le text puis le body

    soup.TEI.append(soup.new_tag("blabla"))
    soup.TEI.blabla.append(soup.new_tag("body"))

    # Importation du fichier xml sous format dataframe

    df3 = pd.read_xml("./xml/" + ark + ".xml")
    df3 = df3.drop('lan', axis=1)
    df3 = df3.drop('regio', axis=1)
    df3 = df3.drop('typ', axis=1)

    # On incorpore les différentes rubriques sous les balises <div>

    for i in reversed(range(1, len(df3.index))):
        list_revue = df3.iloc[i].values.flatten().tolist()

        new = soup.new_tag('div', type=str(list_revue[4]),  n=str(list_revue[0]))
        soup.TEI.blabla.body.insert(0,new)

        new = soup.new_tag('p')
        new.string = str(list_revue[1])
        soup.TEI.blabla.body.div.append(new)

        new = soup.new_tag('docAuthor')
        new.string = str(list_revue[3])
        soup.TEI.blabla.body.div.append(new)

        soup.TEI.blabla.body.find('div')['xml:lang'] = str(list_revue[2])

    #On fait maintenant la manchette

    list_revue = df3.iloc[0].values.flatten().tolist()

    new = soup.new_tag('div', type='manchette',  n=str(list_revue[0]))
    soup.TEI.blabla.body.insert(0,new)

    new = soup.new_tag('p')
    new.string = str(list_revue[1])
    soup.TEI.blabla.body.div.append(new)

    new = soup.new_tag('docAuthor')
    new.string = 'Inconnu'
    soup.TEI.blabla.body.div.append(new)

    soup.TEI.blabla.body.find('div')['xml:lang'] = 'co'

    tag_bla = soup.find('blabla')
    tag_bla.name = 'text'

    # pour finir, on crée un fichier dans le dossier tei/
    
    filename = "AMuvra_N" + str(liste_elements[3]) + "_A" + str(liste_elements[2]) + "_" + ark + ".xml"

    with open("./tei/" + filename, "w") as f:
        f.write(soup.prettify())

---